### Data Ingestion

In [ ]:
# document data strtucture
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Create a sample txt file
import os
os.makedirs("../data/text_files", exist_ok=True)

In [ ]:
sample_text = {
    "../data/text_files/sample1.txt": "This is the content of sample text file 1.",
    "../data/text_files/python_intro.txt": """Introduction to Python Programming

Python is a powerful yet beginner-friendly programming language known for its clean syntax and exceptional versatility. 
It enables developers to build applications faster and with fewer lines of code, making it a favorite across industries. 
From web development and automation to data analytics and artificial intelligence, 
Python provides a strong foundation for solving real-world problems with elegance and efficiency. 
Its massive ecosystem of libraries and frameworks allows you to extend its capabilities into almost any domain.

Python’s popularity continues to grow as businesses and developers embrace its flexibility and future-proof nature. 
Its strong community, continuous updates, and support for cutting-edge technologies make it an ideal language 
for long-term learning and innovation. Whether you're exploring programming for the first time or aiming to build advanced AI systems,
Python empowers you to create, experiment, and scale without limitations."""
}

for filepath, content in sample_text.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)    

print("Sample text files created successfully.")        

In [ ]:
#using text loader to load the text files
from langchain_community.document_loaders import TextLoader
loader = TextLoader("../data/text_files/sample1.txt", encoding="utf-8")
document = loader.load()
print(document)


loader = TextLoader("../data/text_files/python_intro.txt", encoding="utf-8")
document = loader.load()
print(document)


In [ ]:
# Directory Loader to load all text files from a directory
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader(
    "../data/text_files", 
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=False
    )
text_documents = loader.load()
text_documents

In [ ]:
#load PDF files using PyPDFLoader and PyMuPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
loader = DirectoryLoader(
    "../data/pdf", 
    glob="*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=False
    )
pdf_documents = loader.load()

In [ ]:
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [ ]:
chunks=split_documents(pdf_documents)
chunks

### Embeding and Vector Store

In [ ]:
import uuid
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

#embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
class EmbeddingManager:
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model_name = model_name
        self.model =  None
        self._load_model()

    def _load_model(self):
        """
        Any necessary model loading logic.

        """
        self.model = SentenceTransformer(self.model_name)

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts.
        Args:
            texts (List[str]): List of texts to generate embeddings for.
        """
        embeddings = self.model.encode(texts, show_progress_bar=True)
        return embeddings    

#inititalize the Embedding Manager    
embedding_manager = EmbeddingManager()

### Vector Store

In [ ]:
class VectorStore:
    def __init__(self,collection_name: str = "pdf_documents",persist_directory: str = "../data/vector_store"):
        """
        Initialize the VectorStore with ChromaDB settings.
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._intialize_store()

    def _intialize_store(self):
        """
        Initialize ChromaDB client and collection.
        """
        os.makedirs(self.persist_directory, exist_ok=True)
        self.client = chromadb.PersistentClient(path=self.persist_directory)
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF Documents embedding collection for RAG applications"}
        )
        print(f"Vector store initialized with collection: {self.collection_name}")
        print(f"Existing number of documents in the collection: {self.collection.count()}")

    def add_documents(self, documents: List[Any], embeddings: np.array):
        """
        Add documents to the vector store after generating embeddings.
        Args:
            documents (List[Any]): List of Document objects to add.
            embedding_manager (EmbeddingManager): Instance of EmbeddingManager to generate embeddings.
        """
       
        ids =[]
        metadatas =[]
        documents_text =[]
        embeddings_list =[]
        for i, (doc,embedding) in enumerate(zip(documents,embeddings)):
            doc_id = str(uuid.uuid4())
            ids.append(doc_id)
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            documents_text.append(doc.page_content)
            embeddings_list.append(embedding.tolist())

        self.collection.add(
            ids=ids,
            embeddings=embeddings_list,
            metadatas=metadatas,
            documents=documents_text
        )
        print(f"Added {len(documents)} documents to the vector store.")


vector_store = VectorStore()


In [ ]:
# generae embiddings
texts = [doc.page_content for doc in chunks]
vector_embeddings = embedding_manager.generate_embeddings(texts)

#store into vector store
vector_store.add_documents(chunks, vector_embeddings)

### Retrival Pipeline From Vector Store

In [ ]:
class RAGRetriever:
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

        

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """Any
        Generate embedding for the query and retrieve top_k similar documents from the vector store.
        Args:
            query (str): The input query string.
            top_k (int): Number of top similar documents to retrieve.
            score_threshold (float): Minimum similarity score threshold for retrieval.
        """
        print(f"Retrieving documents for query: {query}")
        print(f"Top K: {top_k}, Score Threshold: {score_threshold}")

        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        results = self.vector_store.collection.query(query_embeddings=[query_embedding.tolist()],n_results=top_k) 

        retrieved_docs = []
        if results['documents'] and results['documents'][0]:
            documents = results['documents'][0]
            metadatas = results['metadatas'][0]
            distances = results['distances'][0]
            ids = results['ids'][0]

            for i,(doc_id,document,metadata,distance) in enumerate(zip(ids,documents,metadatas,distances)):
                similarity_score = 1 - distance
                if similarity_score >= score_threshold:
                    retrieved_docs.append({
                        "id": doc_id,
                        "content": document,
                        "metadata": metadata,
                        "similarity_score": similarity_score,
                        "distance": distance,
                        "rank": i + 1       
                    })
            print(f"Retrieved Document {i+1}: ID={doc_id}, Similarity Score={similarity_score:.4f}")
        else:
            print(f"Document ID={doc_id} filtered out due to low similarity score ({similarity_score:.4f})")
        return retrieved_docs
                
        
rag_retriever = RAGRetriever(vector_store, embedding_manager)

In [ ]:
my_query = 'Using Lists as Queues in Python'
vectors = rag_retriever.retrieve(my_query, top_k=5, score_threshold=0.0)
vectors